# _EXPERT_ usage reference: Forecasting with covariates

## Log in to the _EXPERT_ client
Initialize the client. You can either enter your credentials here or store them in the .env file.

If you are using _EXPERT_ for the first time, we recommend that you begin with [Getting Started](simple_workflow.ipynb)

In [ ]:
from futureexpert import (ActualsCovsConfiguration,
                          CovariateRef,
                          DataDefinition,
                          ExpertClient,
                          FileSpecification,
                          ForecastingConfig,
                          PreprocessingConfig,
                          ReportConfig,
                          TsCreationConfig)

client = ExpertClient(user='', password='')

### Configure and run _FORECAST_ with covariates

In some cases, it is advantageous to include additional data as influencing variables in the model. In the example of bicycle data on a daily basis, for demonstration purposes, we show the inclustion of only one covariate &ndash; working days &ndash; although holidays and weather data is increasing the forecast quality as well.

In order to generate a forecast which takes covariates into account, create the covariates first. Define a data definition and a time series creation config similar as for the forecasting data.

The returned identification number must then be added to the report configuration.

You can use the [MATCHER](notebooks/cov_matcher_and_forecast.ipynb) to calculate the best lags for each covariate and create a ranking of each covariate based on their predictive power.

In [ ]:
import futureexpert.checkin as checkin

# Upload Working Days to use them as covariate
covs_check_in_results = client.check_in_time_series(raw_data_source='../example_data/working_days_bavaria.csv',
                                                    data_definition=DataDefinition(date_columns=checkin.DateColumn(name='time', format='%Y-%m-%d'),
                                                                                   value_columns=[checkin.ValueColumn(name='value', name_new='working days bavaria')]),
                                                    config_ts_creation=TsCreationConfig(time_granularity='daily',
                                                                                        value_columns_to_save=['working days bavaria']),
                                                    file_specification=FileSpecification(delimiter=';', decimal=','))


# check in the data you want to forecast
ts_check_in_results = client.check_in_time_series(raw_data_source='../example_data/bicycle_data_single.csv',
                                                  data_definition=DataDefinition(date_columns=checkin.DateColumn(name='date', format='%Y-%m-%d'),
                                                                                 value_columns=[checkin.ValueColumn(name='value', name_new='bicycle count')]),
                                                  config_ts_creation=TsCreationConfig(time_granularity='daily', value_columns_to_save=['bicycle count']))

# Create the covariate configuration
# the name of the actuals and covariate time series can be found in the results from check_in_time_series
covs_configuration = [ActualsCovsConfiguration(actuals_name='bicycle count', covs_configurations=[
                                               CovariateRef(name='working days bavaria', lag=0)])]

fc_report_config = ReportConfig(title='My first report with covariate',
                                covs_versions=[covs_check_in_results.version_id],
                                covs_configuration=covs_configuration,
                                forecasting=ForecastingConfig(fc_horizon=3,
                                                              confidence_level=0.95),
                                preprocessing=PreprocessingConfig(use_season_detection=True,
                                                                  detect_outliers=True,
                                                                  replace_outliers=True)
                                )

forecast_identifier = client.start_forecast(version=ts_check_in_results.version_id, config=fc_report_config)

In [ ]:
import time
# Watch the current status of the forecasting report
while not (current_status := client.get_report_status(id=forecast_identifier)).is_finished:
    current_status.print()
    print('Waiting another 30 seconds to finish forecasting...')
    time.sleep(30)  # Wait between status requests

current_status.print()

# Retrieve the final results
results = client.get_fc_results(id=forecast_identifier, include_backtesting=True, include_k_best_models=100)